# Training a model

> Fill in a module description here

In [ ]:
#| default_exp basic

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

/home/gg/.local/share/virtualenvs/lightning-level-up-mJws7JKj/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export
dataset = MNIST('../data', download=True, transform=transforms.ToTensor())

In [ ]:
! ls ../data/MNIST/raw

t10k-images-idx3-ubyte	   train-images-idx3-ubyte
t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte	   train-labels-idx1-ubyte
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [ ]:
type(dataset)

torchvision.datasets.mnist.MNIST

In [ ]:
#| export
train_loader = DataLoader(dataset, batch_size=10)

In [ ]:
#| export
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Flatten(), nn.Linear(28*28, 64), nn.ReLU(), nn.Linear(64, 3))
        
    def forward(self, x):
        return self.l1(x)

In [ ]:
#| export
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28*28))
        
    def forward(self, x):
        return self.l1(x)

Get a batch:

In [ ]:
xb, yb = next(iter(train_loader))

In [ ]:
xb.shape, yb.shape

(torch.Size([10, 1, 28, 28]), torch.Size([10]))

In [ ]:
encoder_acts = Encoder()(xb)
encoder_acts.shape

torch.Size([10, 3])

In [ ]:
decoder_acts = Decoder()(encoder_acts)
decoder_acts.shape

torch.Size([10, 784])

In [ ]:
out_shape = xb.shape
out_shape

torch.Size([10, 1, 28, 28])

In [ ]:
decoder_acts.view(out_shape).shape

torch.Size([10, 1, 28, 28])

In [ ]:
loss = F.mse_loss(decoder_acts.view(out_shape), xb)
loss

tensor(0.1172, grad_fn=<MseLossBackward0>)

In [ ]:
#| export
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch 
        out_shape = x.shape
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat.view(out_shape), x)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:

# model
autoencoder = LitAutoEncoder(Encoder(), Decoder())

# train model
trainer = pl.Trainer(max_epochs=2)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gg/.local/share/virtualenvs/lightning-level-up-mJws7JKj/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
Missing logger folder: /mnt/data/lightning-level-up/nbs/lightning_logs

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 50.4 K
1 | decoder | Decoder | 51.2 K
------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
/home/gg/.local/share/virtualenvs/lightning-level-up-mJws7JKj/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloa

Epoch 1: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6000/6000 [01:12<00:00, 82.30it/s, loss=0.0396, v_num=0]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6000/6000 [01:12<00:00, 82.28it/s, loss=0.0396, v_num=0]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()